In [1]:
import pymc as pm
import numpy as np

observed, test_point = 5.0, 2.5 

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
with pm.Model() as model:
    z = pm.Normal("z", 0., 5.)
    x = pm.Normal("x", mu=z, sigma=1., observed=observed)

func = model.logp_dlogp_function()
func.set_extra_values({})
ll,gr=func(np.array([test_point]))
print(ll,gr)

-6.697315008914228 [2.4]


In [3]:
import distrax
from jax import grad

def log_likelihood(test_point):
    log_prior=distrax.Normal(0.,5.).log_prob(test_point)
    log_like=distrax.Normal(test_point,1.).log_prob(observed)
    return log_prior+log_like

dlog_p = grad(log_likelihood)

print(log_likelihood(test_point),dlog_p(test_point))

2024-08-20 10:42:11.148248: W external/xla/xla/service/gpu/nvptx_compiler.cc:836] The NVIDIA driver's CUDA version is 12.2 which is older than the PTX compiler version (12.6.20). Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


-6.697315 2.4
